# ipyvolume & bokeh
This example shows how the selection from a ipyvolume quiver plot can be controlled with a bokeh scatter plot and it's selection tools.

## Ipyvolume quiver plot
The 3d quiver plot is done using ipyvolume

In [ ]:
import ipyvolume
import ipyvolume as ipv
import vaex

We load some data from vaex, but only use the first 10 000 samples for performance reasons of Bokeh.

In [ ]:
ds = vaex.example()
N = 10000

We make a quiver plot using ipyvolume's matplotlib's style api.

In [ ]:
ipv.figure()
quiver = ipv.quiver(ds.data.x[:N],  ds.data.y[:N],  ds.data.z[:N],
                    ds.data.vx[:N], ds.data.vy[:N], ds.data.vz[:N],
                    size=1, size_selected=5, color_selected="grey")
ipv.xyzlim(-30, 30)
ipv.show()

## Bokeh scatter part
The 2d scatter plot is done using Bokeh.

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import CustomJS, ColumnDataSource

import ipyvolume.bokeh
output_notebook()

In [ ]:
data_source = ColumnDataSource(data=dict(x=ds.data.Lz[:N], y=ds.data.E[:N]))


In [ ]:
p = figure(title="E Lz space", tools='lasso_select', width=500, height=500)
r = p.circle('x', 'y', source=data_source, color="navy", alpha=0.2)
ipyvolume.bokeh.link_data_source_selection_to_widget(data_source, quiver, 'selected')
show(p)

Now try doing a selection and see how the above 3d quiver plot reflects this selection.

In [ ]:
# but them next to eachother
import ipywidgets
out = ipywidgets.Output()
with out:
    show(p)
ipywidgets.HBox([out, ipv.gcc()])

## Embedding in html

A bit of a hack, but it is possible to embed the widget and the bokeh part into a single html file (use at own risk).

In [ ]:
from bokeh.resources import CDN
from bokeh.embed import components

script, div = components((p))
template_options = dict(extra_script_head=script + CDN.render_js() + CDN.render_css(),
                        body_pre="<h2>Do selections in 2d (bokeh)<h2>" + div + "<h2>And see the selection in ipyvolume<h2>")
ipyvolume.embed.embed_html("tmp/bokeh.html",
                           [ipv.gcc(), ipyvolume.bokeh.wmh], all_states=True,
                           template_options=template_options)

In [1]:
# uncomment the next line to open the html file
# !open tmp/bokeh.html

[screenshot](screenshot/bokeh.png)